In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
import h5py
import seaborn as sns
import pandas as pd
from tensorboard.backend.event_processing import event_accumulator
from scipy.stats import pearsonr
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader

from predify.utils.training import train_pcoders, eval_pcoders

from models.networks_2022 import BranchedNetwork
from data.CleanSoundsDataset import CleanSoundsDataset
from data.NoisyDataset import NoisyDataset, FullNoisyDataset

# PNet parameters

In [13]:
from models.pbranchednetwork_all import PBranchedNetwork_AllSeparateHP
PNetClass = PBranchedNetwork_AllSeparateHP
pnet_name = 'pnet'
chckpt = 50

In [14]:
n_timesteps = 5
layers = ['conv1', 'conv2', 'conv3', 'conv4_W', 'conv5_W', 'fc6_W']

# Paths to relevant directories

In [15]:
engram_dir = '/mnt/smb/locker/abbott-locker/hcnn/'
activations_dir = f'{engram_dir}activations_pnet_all/'
checkpoints_dir = f'{engram_dir}checkpoints/'
tensorboard_dir = f'{engram_dir}tensorboard/'

In [16]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')

Device: cuda:0


# Helper functions to load network

In [17]:
def get_hyperparams(tf_dir, bg, snr, shared=False):
    if shared:
        raise ValueError('Not implemented for shared hyperparameters.')
        
    hyperparams = []
    tf_file_dir = f'{tf_dir}hyper_{bg}_snr{snr}/'
    for tf_file in os.listdir(tf_file_dir):
        tf_file = f'{tf_file_dir}{tf_file}'
        ea = event_accumulator.EventAccumulator(tf_file)
        ea.Reload()
        for i in range(1, 6):
            hps = {}
            ffm = ea.Scalars(f'Hyperparam/pcoder{i}_feedforward')[-1].value
            fbm = ea.Scalars(f'Hyperparam/pcoder{i}_feedback')[-1].value
            erm = ea.Scalars(f'Hyperparam/pcoder{i}_error')[-1].value
            hps['ffm'] = ffm
            hps['fbm'] = fbm
            hps['erm'] = erm
            hyperparams.append(hps)
        break
    return hyperparams

In [18]:
def load_pnet(PNetClass, pnet_name, chckpt, hyperparams=None):
    net = BranchedNetwork(track_encoder_representations=True)
    net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))
    pnet = PNetClass(net, build_graph=False)
    pnet.load_state_dict(torch.load(
        f"{checkpoints_dir}{pnet_name}/{pnet_name}-{chckpt}-regular.pth",
        map_location='cpu'
        ))
    if hyperparams is not None:
        pnet.set_hyperparameters(hyperparams)
    pnet.to(DEVICE)
    pnet.eval();
    print(f'Loaded Pnet: {pnet_name}')
    print_hps(pnet)
    return pnet

In [19]:
def print_hps(pnet):
    for pc in range(pnet.number_of_pcoders):
        print (f"PCoder{pc+1} : ffm: {getattr(pnet,f'ffm{pc+1}'):0.3f} \t fbm: {getattr(pnet,f'fbm{pc+1}'):0.3f} \t erm: {getattr(pnet,f'erm{pc+1}'):0.3f}")

# Helper functions to save activations

In [20]:
n_units_per_layer = {
    'conv1': (96, 55, 134), 'conv2': (256, 14, 34),
    'conv3': (512, 7, 17), 'conv4_W': (1024, 7, 17),
    'conv5_W': (512, 7, 17), 'fc6_W': (4096,)
    }

In [21]:
def run_pnet(pnet, _input):
    pnet.reset()
    reconstructions = []
    activations = []
    logits = []
    output = []
    for t in range(n_timesteps):
        _input_t = _input if t == 0 else None
        logits_t, _ = pnet(_input_t)
        x = pnet.pcoder1.prd
        reconstructions.append(pnet.pcoder1.prd[0,0].cpu().numpy())
        activations.append(pnet.backbone.encoder_repr)
        logits.append(logits_t.cpu().numpy().squeeze())
        output.append(logits_t.max(-1)[1].item())
    return reconstructions, activations, logits, output

In [22]:
@torch.no_grad()
def save_activations(pnet, data, hdf5_path):
    n_data = data.shape[0]
    
    with h5py.File(hdf5_path, 'x') as f_out:
        data_dict = {}
        for layer_idx, layer in enumerate(layers):
            activ_dim = (n_data,) + n_units_per_layer[layer]
            for timestep in range(n_timesteps):
                data_dict[f'{layer}_{timestep}_activations'] = f_out.create_dataset(
                    f'{layer}_{timestep}_activations', activ_dim, dtype='float32'
                    )
        for idx in range(data.shape[0]):
            # Noisy input
            noisy_in = data[idx]
            reconstructions, activations, logits, output = run_pnet(pnet, noisy_in)
            for timestep in range(n_timesteps):
                for layer in layers:
                    data_dict[f'{layer}_{timestep}_activations'][idx] = \
                        activations[timestep][layer]
                    

# Run activation-saving functions

In [23]:
files = ['fg', 'bg_july22']
snrs = [-9.0, -6.0, -3.0, 0.0, 3.0]
tf_dir = f'{tensorboard_dir}lr_0.01x/'

In [24]:
hdf5_path = f'{activations_dir}natural_sounds.hdf5'
#hyperparams = get_hyperparams(tf_dir, bg, snr)
pnet = load_pnet(PNetClass, pnet_name, chckpt, hyperparams=None)
loaded_f = h5py.File(f'{engram_dir}165_natural_sounds_not_resampled.hdf5', 'r')
data = np.array(loaded_f['data']).reshape((-1, 164, 400))
data = torch.tensor(data).to(DEVICE)
save_activations(pnet, data, hdf5_path)

/share/issa/users/es3773/hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/share/issa/users/es3773/hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')


Loaded Pnet: pnet
PCoder1 : ffm: 0.300 	 fbm: 0.300 	 erm: 0.010
PCoder2 : ffm: 0.300 	 fbm: 0.300 	 erm: 0.010
PCoder3 : ffm: 0.300 	 fbm: 0.300 	 erm: 0.010
PCoder4 : ffm: 0.300 	 fbm: 0.300 	 erm: 0.010
PCoder5 : ffm: 0.300 	 fbm: 0.300 	 erm: 0.010


/home/es3773/.conda/envs/hcnn/lib/python3.6/site-packages/torch/nn/functional.py:3680: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default behavior for interpolate/upsample with float scale_factor changed "
/home/es3773/.conda/envs/hcnn/lib/python3.6/site-packages/predify/modules/base.py:260: UserWarning: Using a target size (torch.Size([164, 400])) that is different to the input size (torch.Size([1, 1, 164, 400])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  self.prediction_error  = nn.functional.mse_loss(self.prd, target)
